In [ ]:
#先jieba分词
#从文件夹读取数据
import pandas as pd
import xlrd
fileName = ["aspect_withtime筛选.xlsx"]
alldata = []
for fileName in fileName:
    df = pd.read_excel("./"+fileName)
    data_list = df.values.tolist()
    alldata = alldata+data_list
#去掉表情符号等
import re
# 定义一个包含多个表情符号Unicode区间的正则表达式
emoji_pattern = re.compile(
    u'['    
    u'\U0001F300-\U0001F5FF'  # 符号与表情
    u'\U0001F600-\U0001F64F'  # 表情
    u'\U0001F680-\U0001F6FF'  # 运输和地图符号
    u'\U0001F1E0-\U0001F1FF'  # 区域指示器符号
    u'\u2600-\u27BF'         # 杂项符号
    u'\u24C2-\u24E9'         # 数字符号
    u'\U0001f900-\U0001f9ff'  # 补充符号和图片
    u'\u3030-\u303F'         # 日文假名
    u'\u203C-\u2049'         # 货币和一般标点符号
    u'\u2000-\u206F'         # 一般标点符号
    u'\u2190-\u21FF'         # 箭头
    u'\u2700-\u27BF'         # 括号和标点符号
    u'\u2B00-\u2BFF'         # 几何形状和其他符号
    u'\u3000-\u303F'         # CJK符号和标点
    u'\uFE30-\uFE4F'         # CJK兼容形式
    u'\uFE00-\uFE0F'         # 变异选择器
    u'\u3280-\u32FF'         # 封闭式CJK字母和月份
    u'\u3300-\u33FF'         # CJK兼容形式
    u'\uFFE0-\uFFEF'         # 全角ASCII和半角标点
    u'\u2600-\u26FF'         # 表情符号
    u'\u2700-\u27BF'         # 额外的标点符号
    u'\uE000-\uF8FF'         # 私人使用区域（不可用于公开交换）
    ']+',
    re.UNICODE
)

def remove_emojis(text):
    return emoji_pattern.sub(r'', text)

for data in range(len(alldata)):
    alldata[data][1] = remove_emojis(alldata[data][1])

all_data_4_cut = list(set([data[1] for data in alldata]))


In [ ]:
#jeiba分词
rabit_word = ['',
              ' ',
              '，',
              ',',
              '?',
              '|',
              '（',
              '）',
              '：',
              '！'
              ,'10',
              'Y',
              '感',
              '少',
              '拍',
              '少',
              '玩',
              '走',
              '说',
              '想',
              '高',
              '站',
              '坐',
              '完',
              ]

import jieba
from spacy.lang.zh.stop_words import STOP_WORDS
all_data_4_LDA = []
for line in all_data_4_cut:
    temp_data = list(jieba.cut(line))
    temp_data_2 = [word for word in temp_data if word not in STOP_WORDS]
    temp = [word for word in temp_data_2 if word not in rabit_word]
    all_data_4_LDA.append(temp)

In [ ]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [ ]:
dictionary = corpora.Dictionary(all_data_4_LDA)  # 构建词典
corpus = [dictionary.doc2bow(text) for text in all_data_4_LDA]  #表示为第几个单词出现了几次

In [ ]:
ldamodel = LdaModel(corpus, num_topics=8, id2word = dictionary, passes=30,random_state = 1)   #分为10个主题
print(ldamodel.print_topics(num_topics=8, num_words=15))  #每个主题输出15个单词

In [ ]:
topic_words = []
for i in range(8):
    temp_topics = ldamodel.show_topic(i,topn=len(dictionary))
    topic_words.append(temp_topics)

In [ ]:
topic_words_output = []
for i in range(len(topic_words[0])):
    temp = []
    for j in range(8):
        temp.append(topic_words[j][i][0])
        temp.append(topic_words[j][i][1])
    topic_words_output.append(temp)


In [ ]:
df = pd.DataFrame(topic_words_output)
# 将DataFrame输出到CSV文件
df.to_csv('topic_words_output.csv',header=False, index=False, encoding='utf-8')

In [ ]:
#计算困惑度
def perplexity(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=15))
    print(ldamodel.log_perplexity(corpus))
    return ldamodel.log_perplexity(corpus)
#计算coherence
def coherence(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30,random_state = 1)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    ldacm = CoherenceModel(model=ldamodel, texts=all_data_4_LDA, dictionary=dictionary, coherence='c_v')
    print(ldacm.get_coherence())
    return ldacm.get_coherence()

In [ ]:
x = range(1,14)
y = [perplexity(i) for i in x]  #如果想用困惑度就选这个
# y = [coherence(i) for i in x]
plt.plot(x, y)
plt.xlabel('主题数目')
plt.ylabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()